In [2]:
import http.client, urllib, base64, requests
import cognitive_face as CF 
from cognitive_face import util
import time

In [3]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/Users/Amiros/anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [4]:
#Emotion Analysis parameters
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = 'ead17ccf578a40b88f2bb425d11d4a66' #Here you have to paste your primary key
_maxNumRetries = 5

#Face detection parametrs
CF.Key.set('d78211e12bba41a991382ade5d68745b') #set the key

time: 4.21 ms


In [5]:
# Image path
urlImage= ''

# Emtional Anlaysis function
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None
params = None

time: 46.1 ms


In [6]:
# Face detect function
def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true',
        'returnFaceLandmarks': landmarks and 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

time: 34.1 ms


In [13]:
#Get all the frames from S3
#start with 355
url_list = []
for i in range(689,951,1):
    url = "https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance%d.jpg" % i
    url_list.append(url)

time: 2.77 ms


In [14]:
url_list

['https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance689.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance690.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance691.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance692.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance693.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance694.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance695.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance696.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance697.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance698.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance699.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance700.jpg',
 'https://s3-us-west-2.amazonaws.com/post-election/trump/acceptance701.jpg',

time: 10.2 ms


In [15]:
# Face Detection & Emotion Analysis - UPDATED
processed = {}
# face_detection = {}
count = 689

for single_image in url_list:
 
    try:
        face_detection[count] = detect(single_image)

        json = { 'url': single_image } 
        processed[count] = processRequest(json, data, headers, params)  
    except:
        pass

    try:
        if face_detection[count][0]['faceRectangle'] == processed[count][0]['faceRectangle']:
            face_detection[count][0].update(processed[count][0])
            face_detection[count][1].update(processed[count][1])
        if  face_detection[count][0]['faceRectangle'] == processed[count][1]['faceRectangle']:
            face_detection[count][0].update(processed[count][1])
            face_detection[count][1].update(processed[count][0])
    except:
        pass
    
    count += 1
    
    time.sleep(3)

time: 18min 54s


face_detection

In [ ]:
# Save the result as Json
import json
with open('trump_acceptance.json', 'w') as fp:
    json.dump(face_detection, fp, sort_keys=True)

In [10]:
face_detection

{58: [{'faceAttributes': {'gender': 'male'},
   'faceId': '71865eb4-b308-443f-ade1-ed7ba5314b00',
   'faceRectangle': {'height': 38, 'left': 285, 'top': 54, 'width': 38},
   'scores': {'anger': 0.0287738536,
    'contempt': 0.007499268,
    'disgust': 0.04265538,
    'fear': 0.00242548552,
    'happiness': 0.3181123,
    'neutral': 0.559967935,
    'sadness': 0.03147959,
    'surprise': 0.009086203}}],
 59: [{'faceAttributes': {'gender': 'male'},
   'faceId': '6f44db6f-9ae9-4590-8bc6-366e3b4695e2',
   'faceRectangle': {'height': 37, 'left': 289, 'top': 58, 'width': 37},
   'scores': {'anger': 0.0011179226,
    'contempt': 0.101931646,
    'disgust': 0.00251091365,
    'fear': 9.321558e-05,
    'happiness': 0.395236433,
    'neutral': 0.453221619,
    'sadness': 0.0457076058,
    'surprise': 0.0001806715}}],
 60: [{'faceAttributes': {'gender': 'male'},
   'faceId': 'f85d6174-30f6-422f-8552-f6a54ba0e86f',
   'faceRectangle': {'height': 39, 'left': 285, 'top': 58, 'width': 39},
   'scores

time: 867 ms
